In [24]:
import tensorflow as tf
from tensorflow import keras
import joblib
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import cv2
from server.Wavelet import wavelett_trans

In [19]:
celeb_folders = joblib.load('celeb_folders')

In [25]:
encode_celeb = {}
c = 0

for name in celeb_folders:
    encode_celeb[name] = c
    c += 1
encode_celeb

{'ben_afflek': 0,
 'jerry_seinfeld': 1,
 'lionel_messi': 2,
 'madonna': 3,
 'maria_sharapova': 4,
 'mindy_kaling': 5,
 'roger_federer': 6,
 'serena_williams': 7,
 'virat_kohli': 8}

In [34]:
X = []
Y = []
for celeb, path in celeb_folders.items():
    if not path:
        continue
    for train_data in path:
        img_train = cv2.imread(train_data)
        if img_train is None:
            continue
        scaled_raw_img = cv2.resize(img_train, (32,32))
        img_har = wavelett_trans(img_train, level=5)
        scaled_har_img = cv2.resize(img_har, (32,32))
        stack_img = np.vstack((scaled_raw_img.reshape(32*32*3,1), scaled_har_img.reshape(32*32,1)))
        stack_img = stack_img.reshape(64,64)
        X.append(stack_img)
        Y.append(encode_celeb[celeb])

In [54]:
X = np.array(X, dtype=np.uint8)
Y = np.array(Y, dtype=np.uint8)

In [73]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=20, test_size=0.1)

In [76]:
X_train = X_train/255
X_test = X_test/255

In [57]:
X_train.shape, X_test.shape

((218, 64, 64), (3, 64, 64))

In [77]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(64,64)),
    keras.layers.Dense(2048, activation='relu'),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')
])
opt = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=30)  

Epoch 1/30
7/7 [==============================] - 2s 154ms/step - loss: 2.3926 - accuracy: 0.1111
Epoch 2/30
7/7 [==============================] - 1s 137ms/step - loss: 2.0365 - accuracy: 0.3384
Epoch 3/30
7/7 [==============================] - 1s 136ms/step - loss: 1.9082 - accuracy: 0.3788
Epoch 4/30
7/7 [==============================] - 1s 137ms/step - loss: 1.8013 - accuracy: 0.4242
Epoch 5/30
7/7 [==============================] - 1s 142ms/step - loss: 1.6996 - accuracy: 0.5808
Epoch 6/30
7/7 [==============================] - 1s 148ms/step - loss: 1.6170 - accuracy: 0.6414
Epoch 7/30
7/7 [==============================] - 1s 143ms/step - loss: 1.5386 - accuracy: 0.6667
Epoch 8/30
7/7 [==============================] - 1s 139ms/step - loss: 1.4552 - accuracy: 0.6768
Epoch 9/30
7/7 [==============================] - 1s 140ms/step - loss: 1.3916 - accuracy: 0.6667
Epoch 10/30
7/7 [==============================] - 1s 143ms/step - loss: 1.3280 - accuracy: 0.7475
Epoch 11/30
7/7 [==

In [87]:
model.evaluate(X, Y)

7/7 [==============================] - 0s 16ms/step - loss: 9.4711 - accuracy: 0.9502


[9.471148490905762, 0.9502262473106384]

In [91]:
model_cnn = keras.Sequential([
    keras.layers.Conv2D(input_shape=(64,64,1) ,filters=32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    
    keras.layers.Conv2D(filters= 32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='tanh'),
    keras.layers.Dense(256, activation='tanh'),
    keras.layers.Dense(10, activation='softmax')
])
opt = keras.optimizers.Adam(learning_rate=0.00001)
model_cnn.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_cnn.fit(X_train, Y_train, epochs=20) 

Epoch 1/20
7/7 [==============================] - 2s 194ms/step - loss: 2.2389 - accuracy: 0.1717
Epoch 2/20
7/7 [==============================] - 1s 199ms/step - loss: 2.1296 - accuracy: 0.2121
Epoch 3/20
7/7 [==============================] - 1s 169ms/step - loss: 2.1071 - accuracy: 0.1818
Epoch 4/20
7/7 [==============================] - 1s 165ms/step - loss: 2.0834 - accuracy: 0.1970
Epoch 5/20
7/7 [==============================] - 1s 194ms/step - loss: 2.0754 - accuracy: 0.2424
Epoch 6/20
7/7 [==============================] - 1s 184ms/step - loss: 2.0572 - accuracy: 0.2576
Epoch 7/20
7/7 [==============================] - 1s 174ms/step - loss: 2.0486 - accuracy: 0.1869
Epoch 8/20
7/7 [==============================] - 1s 164ms/step - loss: 2.0382 - accuracy: 0.2172
Epoch 9/20
7/7 [==============================] - 1s 174ms/step - loss: 2.0258 - accuracy: 0.1717
Epoch 10/20
7/7 [==============================] - 1s 190ms/step - loss: 1.9998 - accuracy: 0.3182
Epoch 11/20
7/7 [==

In [89]:
model_cnn.evaluate(X, Y)

7/7 [==============================] - 0s 30ms/step - loss: 0.3050 - accuracy: 0.8914


[0.30495548248291016, 0.8914027214050293]